# **Implementation of SpaRCe model using ReservoirPy library**

---



In [1]:
pip install reservoirpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.6 MB/s eta 0:00:00


# Loading and scaling **MNIST dataset**

In [2]:
from tensorflow.keras.datasets import mnist
import numpy as np

In [3]:
# Načítaj MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizuj dáta
x_train = x_train.reshape(-1, 28 * 28) / 255.0
x_test = x_test.reshape(-1, 28 * 28) / 255.0

# Konverzia tried do one-hot reprezentácie (pre jednoduchšiu prácu s výstupom)
y_train_onehot = np.eye(10)[y_train]
y_test_onehot = np.eye(10)[y_test]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# **Definícia modelu ESN**

In [ ]:
# zatial neskor

# **Krok 2: Inicializácia SpaRCe prahových hodnôt**




In [5]:
# Percentil prahu pre sparsity
n_percentile = 90  # Hodnota sparsity (percentil), nastaviteľná podľa článku
initial_thresholds = np.percentile(np.abs(x_train), n_percentile, axis=0)

# Inicializácia prahov (theta) na základe percentilu
thresholds = np.copy(initial_thresholds)  # Inicializácia prahov pre každý neurón

# **Krok 3: Thresholding SpaRCe**

In [6]:
def apply_threshold(V_tilde, thresholds):     # metóda aplikuje prahy na s aktuálnym stavom rezervoára a vráti X -
    # Vypočítaj x ako sparse variantu výstupu s prahmi
    sign_V = np.sign(V_tilde)
    relu_V = np.maximum(0, np.abs(V_tilde) - thresholds)
    return sign_V * relu_V  # ekvivalent x_i

# Creating a **sparse reservoir**

In [7]:
# Import
from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy import mat_gen

In [8]:
# Nastavenie parametrov ESN modelu
input_dim = 28 * 28  # MNIST obrazky majú 28x28 pixelov
reservoir_size = 500  # počet neurónov v rezervoári
spectral_radius = 0.9  # spektrálny polomer, ktorý kontroluje stabilitu

def generate_sparse_matrix(size, connectivity, radius):
    W = np.random.randn(size, size) * (radius / np.sqrt(size))
    mask = np.random.rand(size, size) < connectivity
    W[~mask] = 0
    return W

# Generovanie náhodnej rezervoárovej matice (W) s požadovanou sparsitou
W = generate_sparse_matrix(reservoir_size, connectivity=0.1, radius=spectral_radius)

# Definícia ESN modelu
reservoir = Reservoir(
    input_dim=input_dim,
    units=reservoir_size,
    W=W,  # nastavíme W podľa požadovaných parametrov
    input_scaling=0.5,  # gamma faktor vstupu
    bias_scaling=0.1,
    lr=0.1  # časová mierka α
)

# Ridge regression pre lineárny výstup
ridge = Ridge(ridge=1e-6)
W_out = np.random.randn(reservoir_size, 10) * 0.1  # výstupná matica W_out

# **Krok 4: Tréning s aktualizáciou prahov**

In [11]:
learning_rate_theta = 0.01  # rýchlosť učenia prahov
learning_rate_W = 0.001  # rýchlosť učenia váh výstupu
num_epochs = 10  # počet epôch

# Gradientný krok pre každý vzor v tréningovej množine
for epoch in range(num_epochs):
    for i, (V_tilde, y_target) in enumerate(zip(x_train, y_train_onehot)):
        # Vypočítame sparse výstup s prahmi
        x_sparse = apply_threshold(V_tilde, thresholds)

        # Chyba a gradient pre váhy výstupu a prahy
        print("W_out.shape:", W_out.shape)
        print("V_tilde.shape:", V_tilde.shape)
        print("x_sparse:", x_sparse.shape)
        y = W_out@V_tilde
        error = y_target - np.dot(W_out, x_sparse)  # chyba výstupu
        dW_out = np.outer(error, x_sparse)  # gradient pre W_out

        # Aktualizácia prahov (theta) podľa gradientného kroku
        delta_theta1 = np.sum(y_target * W_out.T * np.sign(x_sparse), axis=0)
        delta_theta2 = -W_out[y_target.argmax()] * np.sign(x_sparse)

        # Aktualizácia prahov (theta)
        thresholds += learning_rate_theta * (delta_theta1 + delta_theta2)

        # Aktualizácia výstupnej váhovej matice W_out
        W_out += learning_rate_W * dW_out  # aktualizácia váh výstupu


W_out.shape: (500, 10)
V_tilde.shape: (784,)
x_sparse: (784,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 784 is different from 10)

# **Krok 5: Vyhodnotenie modelu**

In [ ]:
# Vyhodnotenie na testovacej množine
correct = 0
for i, V_tilde in enumerate(x_test):
    x_sparse = apply_threshold(V_tilde, thresholds)
    y_pred = np.dot(W_out.T, x_sparse)
    if y_pred.argmax() == y_test[i]:
        correct += 1

accuracy = correct / len(x_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")